# CUNEF MUCD 2021/2022  
## Machine Learning
## Análisis de Siniestralidad de Automóviles

### Autores:
- Andrés Mahía Morado
- Antonio Tello Gómez


In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import ConfusionMatrixDisplay

from xgboost import XGBClassifier

import pickle
import warnings
warnings.filterwarnings('ignore')
%load_ext autotime

time: 0 ns (started: 2021-12-12 22:59:24 +01:00)


In [3]:
from aux_func import evaluate_model

time: 0 ns (started: 2021-12-12 22:59:24 +01:00)


In [4]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")['fatality']
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")['fatality']
xtrain_smote = pd.read_parquet("../data/xtrain_smote.parquet")
ytrain_smote = pd.read_parquet("../data/ytrain_smote.parquet")['fatality']

time: 1.78 s (started: 2021-12-12 22:59:25 +01:00)


# XGBoost

El XGBoost o extreme gradient boosting es un procedimiento para el ensamblado de algoritmos de Machine Learning. Sus siglas hacen referencia a eXtreme Gradient Boosting. Este método, al contrario que el bagging, no procesa en paralelo los modelos, es más complejo.

El procedimiento es secuencial. Se empieza con un modelo, que entrena con unos datos, y se evalua con un conjunto de test. Una vez evaluado el desempeño de ese primer modelo, se localizan aquellas observaciones en las que el modelo ha tenido más problemas y ha fallado. Estas observaciones tienen más probabilidad de aparecer en los datos para el siguiente modelo, y así sucesivamente. Lo que busca es pulir una y otra vez las observaciones que son difíciles de clasificar.

Es un método potente y complejo, flexible en el sentido de que los modelos no tienen por qué ser del mismo tipo.

Procedemos a entrenar un modelo de XGBoost con nuestros datos y parámetros por defecto.

![Highway](https://programmerclick.com/images/517/897f66771ca1b47c9ed91516b58a621d.png)

In [6]:
clf = XGBClassifier(n_jobs=-1, random_state=0)
clf.fit(xtrain, ytrain)

[22:22:19] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

time: 2min 21s (started: 2021-12-12 22:22:15 +01:00)


In [7]:
with open('../models/XGBoost.pickle', 'wb') as f:
    pickle.dump(clf, f)

time: 47 ms (started: 2021-12-12 22:24:36 +01:00)


In [5]:
with open('../models/XGBoost.pickle', 'rb') as f:
    clf = pickle.load(f)

time: 62 ms (started: 2021-12-12 22:59:31 +01:00)


Generamos las predicciones sobre los datos de validación y evaluamos el modelo.

In [6]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8536524588223738
Accuracy of the model: 0.9846430537622728

Classification report: 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    797650
           1       0.56      0.01      0.02     12472

    accuracy                           0.98    810122
   macro avg       0.77      0.51      0.51    810122
weighted avg       0.98      0.98      0.98    810122


time: 2.42 s (started: 2021-12-12 22:59:37 +01:00)


## Ajuste del umbral de predicción

De la misma manera que ocurría en el modelo Random Forest, nuestro modelo obtiene un valor de recall muy bajo para la clase minoritaria.
Procedemos a ajustar el umbral de predicción siguiendo el punto óptimo según la curva ROC.

In [7]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.014000, G-Mean=0.769
ROC-AUC score of the model: 0.8536524588223738
Accuracy of the model: 0.7553183347693311

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.75      0.86    797650
           1       0.05      0.78      0.09     12472

    accuracy                           0.76    810122
   macro avg       0.52      0.77      0.47    810122
weighted avg       0.98      0.76      0.85    810122


time: 1.61 s (started: 2021-12-12 22:59:40 +01:00)


Podemos observar como el ajuste del threshold dota al modelo de un mayor recall para los casos de la clase minoritaria, lo cual nos interesa desde un punto de vista práctico a pesar de reducir la precisión y accuracy del modelo.

## Comprobación de overfitting

Comprobamos si el modelo sufre de overfitting, realizando una predicción sobre la serie de entrenamiento.

In [8]:
ypred = clf.predict(xtrain)
ypred_proba = clf.predict_proba(xtrain)

# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytrain, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytrain,ypred_new_threshold,ypred_proba)

Best Threshold=0.015716, G-Mean=0.781
ROC-AUC score of the model: 0.8670101234095356
Accuracy of the model: 0.78290399470697

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.78      0.88   3192113
           1       0.05      0.78      0.10     48375

    accuracy                           0.78   3240488
   macro avg       0.52      0.78      0.49   3240488
weighted avg       0.98      0.78      0.86   3240488


time: 11.5 s (started: 2021-12-12 22:59:55 +01:00)


A diferencia del modelo Random Forest, XGBoost no ha realizado un ajuste tan pronunciado sobre los datos de entrenamiento.
Esto puede deberse a la naturaleza y proceso interno de los algoritmos, ya que el modelo Random Forest gana robustez cuanto más profundo y optimizado está.
Sin embargo, el modelo XGBoost tiende al overfitting cuando aumentamos el poder de computación.

# XGBoost con SMOTE

Prueba de aplicación del mismo modelo a la versión de los datos que incluye Oversampling de la clase minoritaria mediante SMOTE.

In [11]:
clf = XGBClassifier(n_jobs=-1, random_state=0)
clf.fit(xtrain_smote, ytrain_smote)

[22:26:51] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='approx', validate_parameters=1, verbosity=None)

time: 5min 36s (started: 2021-12-12 22:26:45 +01:00)


In [12]:
with open('../models/XGBoost_smote.pickle', 'wb') as f:
    pickle.dump(clf, f)

time: 47 ms (started: 2021-12-12 22:32:22 +01:00)


In [13]:
with open('../models/XGBoost_smote.pickle', 'rb') as f:
    clf = pickle.load(f)

time: 47 ms (started: 2021-12-12 22:32:22 +01:00)


In [14]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8328945511423933
Accuracy of the model: 0.9843825991640766

Classification report: 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    797650
           1       0.35      0.02      0.03     12472

    accuracy                           0.98    810122
   macro avg       0.67      0.51      0.51    810122
weighted avg       0.98      0.98      0.98    810122


time: 2.26 s (started: 2021-12-12 22:32:22 +01:00)


El modelo sin aplicar el ajuste de threshold no es lo suficientemente descriptivo como para poder compararlo con su versión análoga. Procedemos a realizar dicho ajuste.

## Ajuste del umbral de predicción

In [15]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.019542, G-Mean=0.748
ROC-AUC score of the model: 0.8328945511423933
Accuracy of the model: 0.7343264347838967

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.73      0.84    797650
           1       0.04      0.76      0.08     12472

    accuracy                           0.73    810122
   macro avg       0.52      0.75      0.46    810122
weighted avg       0.98      0.73      0.83    810122


time: 1.53 s (started: 2021-12-12 22:32:25 +01:00)


Tras realizar el ajuste del modelo, podemos afirmar que en nuestro caso concreto y para el tratamiento de los datos que hemos realizado, no existe una razón por la que usar la versión Oversample de los datos.
Los resultados obtenidos son peores respecto del modelo generado con datos sin alterar.